In [29]:
from src.nws import NWSClient, celsius_to_fahrenheit, fahrenheit_to_celsius
from src.params import *
from datetime import datetime, timedelta
import pytz

client = NWSClient()

In [30]:
station = StationID.MIA
radii = {
    StationID.MDW: 8,
    StationID.MIA: 5,
    StationID.NYC: 4,
    StationID.AUS: 5,
    StationID.HOU: 5,
    StationID.DEN: 9,
    StationID.PHL: 6,
}
station_tz = STATION_TZ[station]
prior_days = 0.5

dfs = client.get_timeseries(station, datetime.now(tz=BASE_TZ) - timedelta(days=prior_days), datetime.now(tz=BASE_TZ), radius=radii[station])

for df in dfs:
    df.rename(columns={
        'air_temp_set_1': 'temperature', 
        'relative_humidity_set_1': 'relative_humidity',
        'wind_speed_set_1': 'wind_speed',
        'wind_direction_set_1': 'wind_direction',
        'air_temp_high_6_hour_set_1': 'temperature_high_6_hour',
        'air_temp_high_24_hour_set_1': 'temperature_high_24_hour',
    }, inplace=True)

    df['temperature_f'] = celsius_to_fahrenheit(df['temperature'])

[df.info() for df in dfs]


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 156 entries, 2024-11-22 04:55:00-05:00 to 2024-11-22 16:53:00-05:00
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   temperature              156 non-null    float64
 1   relative_humidity        156 non-null    float64
 2   wind_speed               156 non-null    float64
 3   wind_direction           155 non-null    float64
 4   temperature_high_6_hour  2 non-null      float64
 5   time                     156 non-null    object 
 6   station_id               156 non-null    object 
 7   station_name             156 non-null    object 
 8   distance                 156 non-null    float64
 9   temperature_f            156 non-null    float64
dtypes: float64(7), object(3)
memory usage: 13.4+ KB
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1 entries, 2024-11-22 07:07:00-05:00 to 2024-11-22 07:07:00-05:00
Data columns (total 7 colu

[None, None, None, None, None, None, None, None]

In [31]:
import pandas as pd
from functools import reduce

res_df = pd.concat([df.reset_index() for df in dfs], axis=0, ignore_index=True)
res_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 495 entries, 0 to 494
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype                           
---  ------                    --------------  -----                           
 0   date_time                 495 non-null    datetime64[ns, America/New_York]
 1   temperature               495 non-null    float64                         
 2   relative_humidity         436 non-null    float64                         
 3   wind_speed                436 non-null    float64                         
 4   wind_direction            408 non-null    float64                         
 5   temperature_high_6_hour   2 non-null      float64                         
 6   time                      495 non-null    object                          
 7   station_id                495 non-null    object                          
 8   station_name              495 non-null    object                          
 9   distance  

In [32]:
import pandas as pd
pd.options.plotting.backend = "plotly"

import plotly.express as px

fig = px.line(res_df, x="date_time", y="temperature_f", color="station_name", title=f"Temperature (°F) for Station {station}", width=1800, height=900)
fig.update_traces(mode="lines", hovertemplate=None)
fig.update_layout(hovermode="x")
fig.update_yaxes(title="Temperature (°F)")
fig.update_xaxes(title="Date")

fig.show()